In [15]:
import requests
from PIL import Image
import io
import tensorflow as tf
import torch
import datetime
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path



In [3]:
from datasets import load_dataset

dataset = load_dataset("stochastic/random_streetview_images_pano_v0.0.2")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 11054/11054 [00:26<00:00, 413.90 examples/s]


In [4]:
dataset
# len(dataset['train'])
# dataset['train'][2]['image']
# filtered_data = dataset.filter(lambda example: "x" in example["text"])


DatasetDict({
    train: Dataset({
        features: ['image', 'country_iso_alpha2', 'latitude', 'longitude', 'address'],
        num_rows: 11054
    })
})

In [5]:
country_codes = ["ZA","KR","AR","BW","GR","SK","HK","NL","PE","AU","KH","LT","NZ","RO","MY","SG","AE","FR","ES","IT","IE","LV","IL","JP","CH","AD","CA","RU","NO","SE","PL","TW","CO","BD","HU","CL","IS","BG","GB","US","SI","BT","FI","BE","EE","SZ","UA","CZ","BR","DK","ID","MX","DE","HR","PT","TH"]
country_dict = {}
for i in range(len(country_codes)):
    country_dict[country_codes[i]] = i
    
def country_to_label(country):
    label = [0]*len(country_codes)
    label[country_dict[country]] = 1
    return label

In [11]:
# referenced: https://blog.paperspace.com/convolutional-autoencoder/
# autoencoder classes (CREDIT: LARGELY TAKEN FROM 6_AUTOENCODER NOTEBOOK, but encoder and decoder architectures modified to be convolutional)
# should only have Encoder that has a latent dimension of 50 - corresponding to country weights

class MLPEncoder(torch.nn.Module):

    def __init__(self,
                 number_of_hidden_layers: int,
                 latent_size: int,
                 hidden_size: int,
                 input_size: int,
                 activation: torch.nn.Module):
        """Construct a simple MLP decoder"""

        super().__init__()

        self.latent_size = latent_size
        assert number_of_hidden_layers >= 0, "Decoder number_of_hidden_layers must be at least 0"

        layers = []
        in_channels = 3
        out_channels = 16
        layers.append(torch.nn.Conv2d(in_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=2, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 32 x 32
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=1, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride
                                      padding=1 
                                      ))
        layers.append(activation)
        #16 x 16
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=1,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 8x8 feature maps
        # 64 out channels
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels*4, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # input is now 8x8 feature maps for out_channels*4 channels.
        layers.append(torch.nn.Flatten())
        #flatten to latent_size 
        layers.append(torch.nn.Linear(4*out_channels*8*8 , # features in
                                      latent_size # features out
                                      ))
        layers.append(activation)
        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        #return torch.rand(1,self.latent_size)
        return self.net(x)


In [16]:
# define our training parameters and model
hidden_layers = 4
hidden_size = 30


latent_size = 50

## this might need to change
input_size = 64
lr = 0.001
# lambda weight for classifier's loss
lamb = 1


# fix random seed
torch.manual_seed(0)

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPEncoder( number_of_hidden_layers=hidden_layers,
                 latent_size=latent_size,
                 hidden_size=hidden_size,
                 input_size=input_size,
                 activation=torch.nn.ReLU()).to(device)

# use an optimizer to handle parameter updates
opt = torch.optim.Adam(model.parameters(), lr=lr)

# save all log data to a local directory
run_dir = "logs"

# to clear out TensorBoard and start totally fresh, we'll need to
# remove old logs by deleting them from the directory
!rm -rf ./logs/

# timestamp the logs for each run so we can sort through them
run_time = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")

# initialize a SummaryWriter object to handle all logging actions
logger = SummaryWriter(log_dir=Path(run_dir) / run_time, flush_secs=20)